In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import nltk
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
#title
asins = open('../CategoryCluster/C10/US/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
# ASINs = ASINs[:-1]
len(ASINs)

7080

In [3]:
#synopses
review = open('../CategoryCluster/C10/US/pre/review.csv').read().split('\n')
review[0]
Reviews=[]
for rev in review:
    a = rev.split('|')
    Reviews += a 
Reviews = Reviews[:-1]
len(Reviews)


7080

In [4]:
#ranks
ranks = open('../CategoryCluster/C10/US/pre/ranks.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split('\t')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

7080

In [5]:
# 载入 nltk 的英文停用词作为“stopwords”变量
stopwords = nltk.corpus.stopwords.words('english')

In [6]:
# 载入 nltk 的 SnowballStemmer 作为“stemmer”变量
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [7]:
# 这里我定义了一个分词器（tokenizer）和词干分析器（stemmer），它们会输出给定文本词干化后的词集合
 
def tokenize_and_stem(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
 
def tokenize_only(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [8]:
# 非常不 pythonic，一点也不！
# 扩充列表后变成了非常庞大的二维（flat）词汇表
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Reviews:
    allwords_stemmed = tokenize_and_stem(i) #对每个评价进行分词和词干化
    totalvocab_stemmed.extend(allwords_stemmed) # 扩充“totalvocab_stemmed”列表
 
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

## Tf-idf and document similarity

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# 定义向量化参数
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer(max_df=0.98, max_features=200000,
                                 min_df=0.02, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
%time tf =  vectorizer.fit_transform(Reviews)
%time tfidf = transformer.fit_transform(tf) # 向量化
 

CPU times: user 10.1 s, sys: 53.1 ms, total: 10.1 s
Wall time: 10.2 s
CPU times: user 2.66 ms, sys: 491 µs, total: 3.15 ms
Wall time: 2.8 ms


In [10]:
#获取词袋模型中的前10个词语 
word = vectorizer.get_feature_names()
print(word[:10])

["'m", "'s", 'absolut', 'absolut love', 'actual', 'ador', 'adult', 'amazon', 'ani', 'anim']


In [11]:
# 词袋模型中共有几个词语
len(word)

171

In [12]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在第i类评论中的tf-idf权重  
weight = tfidf.toarray()  
weight

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.5220487,  0.       ,
         0.       ],
       ..., 
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])

In [13]:
weight.shape

(7080, 171)

In [14]:
#词频矩阵
tfidframe = pd.DataFrame(np.round(weight, 2))

In [15]:
tfidframe[:10]

,0,1,2,3,4,5,6,7,8,9,...,161,162,163,164,165,166,167,168,169,170
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.48,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.52,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.31,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


## cosine similarity

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf)
dist = 1 - similarity

In [17]:
print(dist)

[[  0.00000000e+00   1.00000000e+00   8.37024037e-01 ...,   9.35993117e-01
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00  -2.22044605e-16   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  8.37024037e-01   1.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 ..., 
 [  9.35993117e-01   1.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
   -2.22044605e-16   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   0.00000000e+00]]


In [18]:
similarity_df = pd.DataFrame(similarity)
similarity_df[:10]

,0,1,2,3,4,5,6,7,8,9,...,7070,7071,7072,7073,7074,7075,7076,7077,7078,7079
0,1.000000,0.000000,0.162976,0.031906,0.000000,0.000000,0.0,0.0,0.000000,0.023250,...,0.374002,0.000000,0.000000,0.184261,0.0,0.046463,0.000000,0.064007,0.000000,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.000000,0.211308,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.162976,0.000000,1.000000,0.000000,0.000000,0.277217,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.213996,0.000000,0.000000,0.000000
3,0.031906,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.228822,...,0.051154,0.195186,0.243931,0.173158,0.0,0.235526,0.043082,0.060150,0.108867,0.424572
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.274004,0.0,0.0,0.000000,0.000000,...,0.000000,0.169468,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.211308,0.277217,0.000000,0.274004,1.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.159283,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.108635,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.076005,0.187201,0.000000,0.000000
9,0.023250,0.000000,0.000000,0.228822,0.000000,0.000000,0.0,0.0,0.108635,1.000000,...,0.037275,0.000000,0.000000,0.126178,0.0,0.031817,0.000000,0.429865,0.000000,0.000000


## K-means clustering

In [19]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 1.56 s, sys: 64.8 ms, total: 1.63 s
Wall time: 1.46 s
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [20]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [21]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters}
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster'])

In [22]:
# frame[:5]

In [23]:
# frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

In [24]:
# km.inertia_

In [25]:
# grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
# grouped.mean() # 每个聚类的平均排名（0 到 5）

In [26]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
# for i in range(num_clusters):
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print() # 空行
#     print() # 空行

In [27]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# # print(order_centroids)
# for i in range(num_clusters):
#     print("Cluster %d:" % i, end='')
#     time.sleep(1)
#     for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
#         print(' %s' % word[ind]  , end=',')
#     print() # 空行
#     print() # 空行

## LDA

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

In [29]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [30]:
%time lda.fit(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 1min 1s, sys: 256 ms, total: 1min 1s
Wall time: 1min 2s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [31]:
%time lda_matrix = lda.fit_transform(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 57.5 s, sys: 69.9 ms, total: 57.5 s
Wall time: 57.6 s


In [32]:
lda_matrix #表示每一個文檔屬於每一個聚類的概率

array([[ 0.01430437,  0.0143001 ,  0.01428975, ...,  0.32530623,
         0.56034145,  0.0142969 ],
       [ 0.01666679,  0.01667058,  0.01666688, ...,  0.01666702,
         0.84999046,  0.01666667],
       [ 0.01666667,  0.51666288,  0.34999261, ...,  0.01666692,
         0.0166676 ,  0.01666719],
       ..., 
       [ 0.7749655 ,  0.02500288,  0.025008  , ...,  0.02500076,
         0.02500367,  0.02500664],
       [ 0.02      ,  0.02      ,  0.32570468, ...,  0.02000123,
         0.3036255 ,  0.02000131],
       [ 0.05      ,  0.05      ,  0.05      , ...,  0.05001246,
         0.54994683,  0.05003078]])

In [33]:
features = pd.DataFrame(lda_matrix, columns=['T1', 'T2', 'T3','T4','T5','T6','T7','T8','T9','T10'])

In [34]:
features[:5]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.014304,0.014300,0.014290,0.014293,0.014286,0.014287,0.014295,0.325306,0.560341,0.014297
1,0.016667,0.016671,0.016667,0.016669,0.016668,0.016667,0.016669,0.016667,0.849990,0.016667
2,0.016667,0.516663,0.349993,0.016671,0.016668,0.016670,0.016667,0.016667,0.016668,0.016667
3,0.010001,0.010001,0.010002,0.010002,0.320468,0.010002,0.137743,0.010001,0.010003,0.471776
4,0.033333,0.033336,0.699992,0.033333,0.033334,0.033335,0.033333,0.033334,0.033333,0.033336


In [35]:
%time km_lda =km.fit_transform(features)

CPU times: user 204 ms, sys: 10.7 ms, total: 215 ms
Wall time: 206 ms


In [36]:
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])

In [37]:
ldaframe = pd.DataFrame({ 'title': ASINs, 'rank': Ranks, 'review': Reviews})
ldaframe = ldaframe[['title','rank','review']]
ldaClusterframe = pd.concat([ldaframe, cluster_labels], axis=1)
ldaClusterframe[:5]

,title,rank,review,ClusterLabel
0,﻿B0756TN4VV_1,5.0,Santa came through! Thank you for a reasonable...,7
1,B0756TN4VV_2,5.0,My great-niece was THRILLED to receive this at...,7
2,B0756TN4VV_3,5.0,ridiculously over priced but my granddaughter ...,8
3,B0756TN4VV_4,5.0,Love these! My 6 year and teenagers like them!...,9
4,B0756TN4VV_5,5.0,Great items onve you dont pay more than $10 fo...,5


In [38]:
ldaClusterframe['ClusterLabel'].value_counts()

4    950
2    917
3    890
0    844
7    770
8    662
9    554
1    529
5    483
6    481
Name: ClusterLabel, dtype: int64

In [39]:
Rankgrouped = ldaClusterframe['rank'].groupby(ldaClusterframe['ClusterLabel']) # 为了凝聚（aggregation），由聚类分类。
 
Rankgrouped.mean() # 每个聚类的平均排名（0 到 5）

ClusterLabel
0    4.622038
1    4.843100
2    4.322792
3    4.931461
4    4.227368
5    4.726708
6    4.261954
7    4.033766
8    4.836858
9    4.608303
Name: rank, dtype: float64

In [40]:
ldaClusterframe['rank'].describe()

count    7080.000000
mean        4.523446
std         1.052923
min         0.000000
25%         5.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: rank, dtype: float64

In [41]:
#排序
ldaClusterframe.sort_values(by = 'ClusterLabel')[:10]

,title,rank,review,ClusterLabel
2650,B01F0HKE1M_1,5.0,I was nervous buying it after seeing old revie...,0
3369,1223104834_10,5.0,"I bought this as a birthday gift. Honestly, I ...",0
698,B07583WCMJ_9,5.0,Pikmi pops are SO fun and really high quality ...,0
5672,B00B4WJ1OO_3,5.0,My animal-loving son was over-the-top excited ...,0
701,B00SUEEMPC_2,5.0,much bigger than i thought it would be but its...,0
5675,B00B4WJ1OO_6,5.0,Very cute and well made. I would highly recom...,0
5677,B00B4WJ1OO_8,5.0,Looks very lifelike. It is a gift for my niece.,0
6516,B01N39LX3X_7,4.0,Outrageously priced it's obvious to see but my...,0
5378,B01IO959R2_9,5.0,This arrived with nightmare Foxy and is just a...,0
5679,B00B4WJ1OO_10,5.0,This little squirrel is very cute in person.,0


In [42]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
#     print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        
        
    print() # 空行
    print() # 空行

 love, daughter, perfect, size, daughter love, 's, bear, babi, just, grandson, ador, littl, perfect size, soft, carri, birthday, small, big, plush, bought, use, right, face, sleep, new, thought, nice, real, cudd, present,

 love, kid, granddaught, gift, cat, christma, got, bought, kid love, great, granddaught love, pretti, want, happi, pusheen, buy, littl, gave, sure, ador, soft, month, onli, just, 'm, open, eye, face, anoth, place,

 cute, great, realli, price, think, item, littl, ador, girl, like, doll, lot, soft, plush, realli cute, babi, perfect, expect, fun, color, 's, bought, worth, nice, say, cudd, play, place, child, collect,

 work, son, time, littl, love, use, batteri, guy, t, make, just, long, arriv, christma, son love, did, day, came, awesom, got, tri, new, talk, hold, toy, quick, need, present, ani, worth,

 's, n't, like, play, toy, doe, just, onli, littl, doe n't, realli, month, got, make, becaus, thing, sleep, fun, sinc, say, month old, 'm, daughter, look, head, want, a

In [43]:
order_centroids

array([[ 89,  37, 106, ..., 128, 142,  82],
       [ 89,  81,  67, ...,  91, 107, 140],
       [ 36,  70, 122, ...,  26,  82, 102],
       ..., 
       [  9, 144, 146, ..., 128, 102,  68],
       [ 94, 118,  87, ...,  93,  68, 102],
       [157,  89, 128, ...,  71,  68,  48]])

In [44]:
ftf = transformer.fit_transform(order_centroids)
ftf = ftf.toarray()

In [45]:
#词频矩阵
tfidMMframe = pd.DataFrame(np.round(ftf, 2))
tfidMMframe[:5]

,0,1,2,3,4,5,6,7,8,9,...,161,162,163,164,165,166,167,168,169,170
0,0.07,0.03,0.08,0.10,0.03,0.00,0.01,0.01,0.06,0.05,...,0.13,0.08,0.09,0.01,0.10,0.11,0.04,0.10,0.11,0.06
1,0.07,0.06,0.05,0.05,0.02,0.02,0.05,0.02,0.06,0.05,...,0.05,0.11,0.11,0.13,0.08,0.05,0.04,0.07,0.08,0.11
2,0.03,0.05,0.09,0.09,0.12,0.06,0.07,0.00,0.05,0.07,...,0.05,0.03,0.05,0.13,0.13,0.05,0.02,0.02,0.06,0.08
3,0.13,0.11,0.12,0.07,0.07,0.12,0.01,0.06,0.12,0.07,...,0.06,0.13,0.09,0.13,0.11,0.04,0.08,0.05,0.07,0.05
4,0.00,0.07,0.06,0.09,0.12,0.03,0.06,0.08,0.07,0.04,...,0.13,0.06,0.05,0.06,0.12,0.08,0.11,0.11,0.02,0.05


In [46]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
    print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        print(' %f' % lda.components_[i][ind-1], end=',')
        print(' %f' % (tfidMMframe[i:i+1][ind]), end=',')#a词在所属群中出现的频率
        print(' %f' % max(tfidframe[:][ind]), end=',') #a词与所有顾客评论中出现最高词频的相对指数
        print(' %f' % (tfidMMframe[i:i+1][ind]/max(tfidframe[:][ind])), end='\n')
        
    print('') # 空行
    print() # 空行

Cluster 0:
 love, 0.100010, 0.110000, 1.000000, 0.110000
 daughter, 0.566175, 0.100000, 0.710000, 0.140845
 perfect, 0.100017, 0.090000, 1.000000, 0.090000
 size, 0.100021, 0.010000, 1.000000, 0.010000
 daughter love, 931.657243, 0.020000, 0.740000, 0.027027
 's, 0.100022, 0.030000, 0.830000, 0.036145
 bear, 0.100004, 0.020000, 0.950000, 0.021053
 babi, 0.100017, 0.110000, 0.910000, 0.120879
 just, 0.100008, 0.040000, 1.000000, 0.040000
 grandson, 0.100005, 0.080000, 1.000000, 0.080000
 ador, 9.514281, 0.000000, 1.000000, 0.000000
 littl, 0.100015, 0.130000, 0.840000, 0.154762
 perfect size, 640.144405, 0.110000, 0.670000, 0.164179
 soft, 0.100016, 0.040000, 1.000000, 0.040000
 carri, 0.100016, 0.030000, 1.000000, 0.030000
 birthday, 103.452143, 0.120000, 0.790000, 0.151899
 small, 66.257873, 0.120000, 1.000000, 0.120000
 big, 0.100026, 0.020000, 0.720000, 0.027778
 plush, 0.115587, 0.080000, 0.840000, 0.095238
 bought, 14.596378, 0.040000, 0.720000, 0.055556
 use, 0.100012, 0.110000, 

 old, 79.583734, 0.040000, 0.640000, 0.062500
 year, 12.805072, 0.030000, 0.580000, 0.051724
 year old, 923.185259, 0.110000, 0.610000, 0.180328
 love, 0.100008, 0.120000, 1.000000, 0.120000
 bought, 0.100017, 0.120000, 0.720000, 0.166667
 old love, 1075.172029, 0.030000, 0.720000, 0.041667
 absolut, 0.100026, 0.130000, 0.880000, 0.147727
 doll, 22.116459, 0.120000, 1.000000, 0.120000
 christma, 0.100020, 0.050000, 0.660000, 0.075758
 absolut love, 148.385247, 0.070000, 0.640000, 0.109375
 son, 0.100022, 0.060000, 0.640000, 0.093750
 hit, 0.100009, 0.050000, 0.850000, 0.058824
 daughter, 50.435867, 0.040000, 0.710000, 0.056338
 everi, 7.040484, 0.040000, 0.600000, 0.066667
 sleep, 0.100015, 0.050000, 0.700000, 0.071429
 night, 67.251921, 0.020000, 0.620000, 0.032258
 big, 0.100014, 0.010000, 0.720000, 0.013889
 toy, 61.955810, 0.090000, 1.000000, 0.090000
 niec, 21.743814, 0.020000, 0.920000, 0.021739
 becaus, 0.100018, 0.010000, 0.630000, 0.015873
 gave, 0.100012, 0.070000, 0.700000, 

In [47]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      17.271449        1       1  0.060281 -0.108079
5      12.028633        1       2  0.264044  0.045054
8      11.104687        1       3  0.140795 -0.298641
7      10.750271        1       4  0.064051  0.047350
0      10.369212        1       5 -0.125644  0.171113
3       8.935379        1       6 -0.152652 -0.141764
6       7.967981        1       7 -0.256221  0.006889
1       7.559647        1       8 -0.150948 -0.044686
2       7.199960        1       9  0.176851  0.178041
9       6.812782        1      10 -0.020557  0.144723, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
161   Default  1816.000000           veri  1816.000000  30.0000  30.0000
89    Default  3987.000000           love  3987.000000  29.0000  29.0000
101   Default  1187.000000            old  1187.000000  28.0000  28.0000
169   Default   957.000000           year   957.000000  27.0000  27.0000
157   Default  1506.000000            toy  1506.000000  26.0000  26.0000
170   Default   794.000000       year old   794.000000  25.0000  25.0000
36    Default  2114.000000           cute  2114.000000  24.0000  24.0000
37    Default  1282.000000       daughter  1282.000000  23.0000  23.0000
94    Default  1509.000000            n't  1509.000000  22.0000  22.0000
9     Default   648.000000           anim   648.000000  21.0000  21.0000
81    Default   676.000000            kid   676.000000  20.0000  20.0000
61    Default  1145.000000           gift  1145.000000  19.0000  19.0000
144   Default   680.000000           stuf   680.000000  18.0000  18.0000
67    Default   496.000000    granddaught   496.000000  17.0000  17.0000
146   Default   563.000000          super   563.000000  16.0000  16.0000
106   Default   855.000000        perfect   855.000000  15.0000  15.0000
70    Default  1124.000000          great  1124.000000  14.0000  14.0000
122   Default   802.000000         realli   802.000000  13.0000  13.0000
119   Default   740.000000        qualiti   740.000000  12.0000  12.0000
38    Default   509.000000  daughter love   509.000000  11.0000  11.0000
1     Default  2213.000000             's  2213.000000  10.0000  10.0000
133   Default   687.000000           size   687.000000   9.0000   9.0000
29    Default   425.000000            cat   425.000000   8.0000   8.0000
45    Default   572.000000            doe   572.000000   7.0000   7.0000
145   Default   448.000000      stuf anim   448.000000   6.0000   6.0000
167   Default   495.000000           work   495.000000   5.0000   5.0000
31    Default   782.000000       christma   782.000000   4.0000   4.0000
115   Default   560.000000          price   560.000000   3.0000   3.0000
139   Default   721.000000            son   721.000000   2.0000   2.0000
128   Default   390.000000              s   390.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
47    Topic10   160.321619            dog   217.321383   2.3822  -3.4896
157   Topic10   741.678546            toy  1506.191937   1.9779  -1.9579
52    Topic10   234.394730         expect   480.748287   1.9680  -3.1098
30    Topic10   166.919097          child   356.164320   1.9285  -3.4493
55    Topic10    83.934852        favorit   205.242956   1.7922  -4.1368
166   Topic10    74.674713            way   193.101055   1.7363  -4.2537
155   Topic10    89.649211        thought   286.972433   1.5229  -4.0709
97    Topic10    78.302234            new   250.759567   1.5225  -4.2062
135   Topic10   138.116461          small   495.076794   1.4098  -3.6387
64    Topic10   175.338962           good   662.056461   1.3577  -3.4001
88    Topic10    62.850581            lot   238.355493   1.3534  -4.4261
153   Topic10   103.840306          thing   395.247185   1.3497  -3.9240
6     Topic10    45.3251

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 14:08:20.385662. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [48]:
# #挑選特定條件(抽樣)
# samples = ldaClusterframe[ldaClusterframe.loc[:,'ClusterLabel'] == 4][:441]
# Rsamples = samples['review'].values
# Rsamples

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 14:08:22.237155. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [49]:
# with open('../test.csv','w',encoding='utf-8') as f :
#     f.write(str(Rsamples))

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 14:08:22.245023. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
